In [1]:
import datetime as dt
from utility import (
    read_sql,
    fd_alive_funds,
    fd_basicinfo,
    fd_typeclass,
    fd_hshkiport,
    fd_assetportfolio,
    fd_qtfdnav,
    nearest_report_date
)
import pandas as pd
from PyFin.api import makeSchedule
from PyFin.api import BizDayConventions

# 1. 基金分类
-----------------------

In [2]:
# 获取回溯的报告日

today = dt.datetime.today()
today = dt.date(2015, 1, 30)
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [3]:
# 获取所有目标混合型基金（清算结束日期晚于当前日）

security_ids = fd_alive_funds(report_dates_begin, 202)

In [4]:
# 获取相关组合情况

basic_info = fd_basicinfo(security_ids, current_date)
type_info = fd_typeclass(security_ids, current_date)
hkport_info = fd_hshkiport(security_ids, report_dates_begin, current_date)
hkport_info = hkport_info[hkport_info.INDUSTRYNAME == "合计"].groupby(["SECURITYID"], as_index=False)[["ACCNETMKTCAP"]].mean()
asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).groupby("SECURITYID", as_index=False)[["EQUITYINVERTO"]].mean()

In [5]:
df = pd.merge(basic_info, type_info, on="SECURITYID")
df = pd.merge(df, asset_port, on="SECURITYID", how="left")
df = pd.merge(df, hkport_info, on="SECURITYID", how="left")
if "ACCNETMKTCAP" in df:
    df["ACCNETMKTCAP"] = df["ACCNETMKTCAP"].fillna(0)
else:
    df["ACCNETMKTCAP"] = 0.0

In [6]:
# 分类
flag = df["ACCNETMKTCAP"] / df["EQUITYINVERTO"] > 0.5
df.loc[flag, "混合型（子类）"] = "港股通混合型基金"
df.loc[(~flag) & (df["EQUITYINVERTO"] >= 70), "混合型（子类）"] = "高权益仓位混合型基金"
df.loc[((~flag) & (df["EQUITYINVERTO"] >= 50) & (df["EQUITYINVERTO"] < 70)), "混合型（子类）"] = "中高权益仓位混合型基金"
df.loc[((~flag) & (df["EQUITYINVERTO"] >= 30) & (df["EQUITYINVERTO"] < 50)), "混合型（子类）"] = "中权益仓位混合型基金"
df.loc[((~flag) & (df["EQUITYINVERTO"] >= 0) & (df["EQUITYINVERTO"] < 30)), "混合型（子类）"] = "低权益仓位混合型基金"
df = df.sort_values("SECURITYID").dropna(subset=["混合型（子类）"])

In [7]:
df

,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,EQUITYINVERTO,ACCNETMKTCAP,混合型（子类）
0,1030000006,华夏成长证券投资基金,None,000001,证券投资基金,成长型,2,混合基金,2.1,偏股型基金,2.1.2,偏股型基金(股票上限80%),65.283333,0.0,中高权益仓位混合型基金
1,1030000007,华夏大盘精选证券投资基金,华夏大盘精选混合,000011,证券投资基金,增值型,2,混合基金,2.1,偏股型基金,2.1.1,偏股型基金(股票上限95%),82.620833,0.0,高权益仓位混合型基金
2,1030000016,华夏回报证券投资基金,华夏回报混合,002001,证券投资基金,收益型,2,混合基金,2.9,特定策略混合型基金,2.9.1,特定策略混合型基金,56.847500,0.0,中高权益仓位混合型基金
3,1030000016,华夏回报证券投资基金,华夏回报混合,002001,证券投资基金,收益型,2,混合基金,2.6,绝对收益目标基金,2.6.3,灵活策略基金（A类）,56.847500,0.0,中高权益仓位混合型基金
4,1030000017,华夏红利混合型证券投资基金,None,002011,证券投资基金,分红型,2,混合基金,2.2,灵活配置型基金,2.2.3,灵活配置型基金(股票上限95%)（A类）,83.277500,0.0,高权益仓位混合型基金
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2030000080,泰达宏利效率优选混合型证券投资基金(LOF),None,162207,LOF,价值投资型,2,混合基金,2.1,偏股型基金,2.1.2,偏股型基金(股票上限80%),68.766667,0.0,中高权益仓位混合型基金
172,2030000084,摩根士丹利华鑫资源优选混合型证券投资基金(LOF),None,163302,LOF,稳健成长型,2,混合基金,2.1,偏股型基金,2.1.1,偏股型基金(股票上限95%),77.436667,0.0,高权益仓位混合型基金
91,2030000085,兴全趋势投资混合型证券投资基金,None,163402,LOF,价值投资型,2,混合基金,2.1,偏股型基金,2.1.1,偏股型基金(股票上限95%),75.867500,0.0,高权益仓位混合型基金
183,2030000087,中银中国精选混合型开放式证券投资基金,None,163801,LOF,增值型,2,混合基金,2.1,偏股型基金,2.1.1,偏股型基金(股票上限95%),68.943333,0.0,中高权益仓位混合型基金


## 1.1 Put it all totether

In [8]:
def create_mutua_fund_info(trade_dt):
    trade_dt = dt.datetime.strptime(trade_dt, "%Y%m%d")
    date_3yrs_ago = trade_dt.replace(year=trade_dt.year - 3)
    report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
    current_date = trade_dt.strftime("%Y%m%d")
    
    security_ids = fd_alive_funds(report_dates_begin, 202)
    basic_info = fd_basicinfo(security_ids, current_date)
    type_info = fd_typeclass(security_ids, current_date)
    hkport_info = fd_hshkiport(security_ids, report_dates_begin, current_date)
    hkport_info = hkport_info[hkport_info.INDUSTRYNAME == "合计"].groupby(["SECURITYID"], as_index=False)[["ACCNETMKTCAP"]].mean()
    asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).groupby("SECURITYID", as_index=False)[["EQUITYINVERTO"]].mean()

    df = pd.merge(basic_info, type_info, on="SECURITYID")
    df = pd.merge(df, asset_port, on="SECURITYID", how="left")
    df = pd.merge(df, hkport_info, on="SECURITYID", how="left").drop_duplicates()
    if "ACCNETMKTCAP" in df:
        df["ACCNETMKTCAP"] = df["ACCNETMKTCAP"].fillna(0)
    else:
        df["ACCNETMKTCAP"] = 0.0
    
    flag = df["ACCNETMKTCAP"] / df["EQUITYINVERTO"] > 0.5
    df.loc[flag, "混合型（子类）"] = "港股通混合型基金"
    df.loc[(~flag) & (df["EQUITYINVERTO"] >= 70), "混合型（子类）"] = "高权益仓位混合型基金"
    df.loc[((~flag) & (df["EQUITYINVERTO"] >= 50) & (df["EQUITYINVERTO"] < 70)), "混合型（子类）"] = "中高权益仓位混合型基金"
    df.loc[((~flag) & (df["EQUITYINVERTO"] >= 30) & (df["EQUITYINVERTO"] < 50)), "混合型（子类）"] = "中权益仓位混合型基金"
    df.loc[((~flag) & (df["EQUITYINVERTO"] >= 0) & (df["EQUITYINVERTO"] < 30)), "混合型（子类）"] = "低权益仓位混合型基金"
    
    return df.sort_values("SECURITYID").dropna(subset=["混合型（子类）"])

In [14]:
%%time

create_mutua_fund_info("20211111")#.groupby( "混合型（子类）").count()

Wall time: 8.01 s


,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,EQUITYINVERTO,ACCNETMKTCAP,混合型（子类）
0,1030000001,华夏复兴混合型证券投资基金,None,000031,证券投资基金,稳健成长型,2,混合基金,2.1,偏股型基金,2.1.3,偏股型基金（股票上下限60%-95%）,89.665833,0.0,高权益仓位混合型基金
1767,1030000004,南方盛元红利混合型证券投资基金,None,202009,证券投资基金,稳健成长型,2,混合基金,2.1,偏股型基金,2.1.3,偏股型基金（股票上下限60%-95%）,88.608333,0.0,高权益仓位混合型基金
2025,1030000005,工银瑞信红利混合型证券投资基金,None,481006,证券投资基金,稳健成长型,2,混合基金,2.1,偏股型基金,2.1.3,偏股型基金（股票上下限60%-95%）,92.279167,0.0,高权益仓位混合型基金
1,1030000006,华夏成长证券投资基金,None,000001,证券投资基金,成长型,2,混合基金,2.1,偏股型基金,2.1.2,偏股型基金(股票上限80%),71.650833,0.0,高权益仓位混合型基金
2,1030000007,华夏大盘精选证券投资基金,华夏大盘精选混合,000011,证券投资基金,增值型,2,混合基金,2.1,偏股型基金,2.1.1,偏股型基金(股票上限95%),87.699167,0.0,高权益仓位混合型基金
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658,2030007999,财通福盛多策略混合型发起式证券投资基金(LOF),None,501032,LOF,稳健成长型,8,封闭式混合基金,8.10,封闭式其他混合型基金,8.10.1,封闭式其他混合型基金,56.721538,0.0,中高权益仓位混合型基金
1503,2030008078,国投瑞银瑞泰多策略灵活配置混合型证券投资基金(LOF),国投瑞银瑞泰多策略混合(LOF),161233,LOF,稳健成长型,8,封闭式混合基金,8.10,封闭式其他混合型基金,8.10.1,封闭式其他混合型基金,35.731667,0.0,中权益仓位混合型基金
2111,2030008355,银华惠安定期开放混合型证券投资基金,None,501033,LOF,稳健成长型,8,封闭式混合基金,8.4,封闭式偏债型基金,8.4.1,封闭式偏债型基金,3.400000,0.0,低权益仓位混合型基金
2106,2030008438,平安鼎弘混合型证券投资基金(LOF),平安鼎弘混合(LOF),167003,LOF,稳健成长型,8,封闭式混合基金,8.4,封闭式偏债型基金,8.4.1,封闭式偏债型基金,14.785000,0.0,低权益仓位混合型基金


# 2. 基金指数
-----------------

In [10]:
# 我们只计算指数的每日收益
# 在确定指数的基准日之后，可以直接使用收益计算指数的值。
# 使用 fd_qtfdnav 获取基金的净值情况

start_dt = "2015-01-31"
final_trade_dt = "2021-11-09"
nav_type = "REPAIRUNITNAV"

rebalance_dates = [d.strftime("%Y%m%d") for d in makeSchedule(start_dt, final_trade_dt, tenor="3M", calendar="china.sse", dateRule=BizDayConventions.ModifiedFollowing)]

In [11]:
dfs = []

for i, trade_dt in enumerate(rebalance_dates[1:]):
    pre_trade_dt = rebalance_dates[i]
    print(pre_trade_dt, trade_dt)
    fund_info = create_mutua_fund_info(trade_dt)
    pre_nav_info = fd_qtfdnav(fund_info.SECURITYID.tolist(), pre_trade_dt).rename(columns={nav_type: "PRE" + nav_type})
    nav_info = fd_qtfdnav(fund_info.SECURITYID.tolist(), trade_dt)

    total_df = pd.merge(fund_info, nav_info, on=["SECURITYID"])
    total_df = pd.merge(total_df, pre_nav_info, on=["SECURITYID"])
    total_df["chg."] = total_df[nav_type] / total_df["PRE" + nav_type] - 1.0
    res = total_df.groupby("混合型（子类）")[["chg."]].mean()
    dfs.append(res)

20150130 20150430
20150430 20150731
20150731 20151030
20151030 20160129
20160129 20160429
20160429 20160729
20160729 20161031
20161031 20170126
20170126 20170428
20170428 20170731
20170731 20171031
20171031 20180131
20180131 20180427
20180427 20180731
20180731 20181031
20181031 20190131
20190131 20190430
20190430 20190731
20190731 20191031
20191031 20200123
20200123 20200430
20200430 20200731
20200731 20201030
20201030 20210129
20210129 20210430
20210430 20210730
20210730 20211029
20211029 20211109


In [12]:
final_report = pd.concat(dfs, keys=rebalance_dates[1:]).reset_index()
final_report.pivot_table(index="level_0", columns="混合型（子类）", values="chg.").to_excel("020_混合型基金_bak.xlsx")

In [13]:
final_report.pivot_table(index="level_0", columns="混合型（子类）", values="chg.")

混合型（子类）,中权益仓位混合型基金,中高权益仓位混合型基金,低权益仓位混合型基金,港股通混合型基金,高权益仓位混合型基金
level_0,,,,,
20150430,0.102836,0.341442,0.087205,NaN,0.397004
20150731,-0.018591,-0.069235,-0.016086,NaN,-0.119116
20151030,0.004786,0.009111,0.011513,NaN,-0.009433
20160129,-0.038824,-0.117369,-0.004973,NaN,-0.161743
20160429,0.040956,0.071110,0.011349,NaN,0.081925
20160729,0.015070,0.033349,0.020062,NaN,0.035694
20161031,0.015578,0.023562,0.009353,NaN,0.022629
20170126,-0.014774,-0.026624,-0.007219,NaN,-0.045320
20170428,0.002694,0.033091,0.011402,NaN,0.030288
